# GuardReasoner R-SFT Training on T4 Colab

**Based on:** Liu et al. "GuardReasoner: Towards Reasoning-based LLM Safeguards" (2025)

**What this does:**
- Fine-tunes LLaMA 3.2-3B on reasoning-based safety classification
- Uses GuardReasonerTrain dataset (127K samples with reasoning traces)
- Implements R-SFT (Reasoning Supervised Fine-Tuning)
- Runs on free T4 Colab GPU

**Expected result:** Model that reasons before classifying (like GuardReasoner paper)

---

**Runtime:** ~3-4 hours on T4 for 1 epoch

**GPU:** Free Tesla T4 (15GB VRAM)

**Output:** Fine-tuned model for safety reasoning

## 1. Installation

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Colab-specific installation
    import torch; v = re.match(r"[0-9\.]{3,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

## 2. Load Base Model (LLaMA 3.2-3B)

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Reasoning traces can be long
dtype = None  # Auto-detect (Float16 for T4)
load_in_4bit = True  # 4-bit quantization for T4 GPU

print("Loading LLaMA 3.2-3B-Instruct...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("✓ Model loaded successfully!")

## 3. Add LoRA Adapters

**GuardReasoner settings:**
- LoRA rank: 16 (paper uses this)
- Target modules: All attention + MLP layers
- Updates ~1-2% of parameters (efficient!)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,  # LoRA rank (GuardReasoner uses 16)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,  # Optimized for unsloth
    bias = "none",
    use_gradient_checkpointing = "unsloth",  # Save VRAM
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("✓ LoRA adapters added!")

## 4. Load GuardReasoner Training Dataset

**Dataset:** 127K samples with reasoning traces from 4 sources:
- WildGuardTrainR: 86,759 samples
- BeaverTailsTrainR: 27,186 samples
- AegisTrainR: 10,798 samples
- ToxicChatTrainR: 2,801 samples

In [ ]:
from datasets import load_dataset

print("Downloading GuardReasonerTrain dataset...")
print("This may take 2-3 minutes (372MB download)")

# Load all splits
dataset_dict = load_dataset("yueliu1999/GuardReasonerTrain")

# Combine all splits into one training set
from datasets import concatenate_datasets
train_dataset = concatenate_datasets([
    dataset_dict["WildGuardTrainR"],
    dataset_dict["AegisTrainR"],
    dataset_dict["BeaverTailsTrainR"],
    dataset_dict["ToxicChatTrainR"],
])

print(f"✓ Dataset loaded: {len(train_dataset):,} samples")
print(f"\nSample structure:")
print(f"  Columns: {train_dataset.column_names}")
print(f"\nFirst sample preview:")
sample = train_dataset[0]
print(f"  Instruction length: {len(sample['instruction'])} chars")
print(f"  Input length: {len(sample['input'])} chars")
print(f"  Output length: {len(sample['output'])} chars (reasoning + answer)")

### View Sample Reasoning Trace

In [ ]:
print("="*80)
print("EXAMPLE: How GuardReasoner Training Data Looks")
print("="*80)

sample = train_dataset[100]  # Pick a clean example

print("\n[INSTRUCTION]")
print(sample['instruction'][:200] + "...")

print("\n[INPUT - User Prompt]")
print(sample['input'][:300] + "...")

print("\n[OUTPUT - Reasoning + Classification]")
print(sample['output'][:600] + "...")

print("\n" + "="*80)
print("Key insight: Model learns to REASON before deciding!")
print("="*80)

## 5. Format Dataset for Training

**Convert to chat format:**
- instruction → system message
- input → user message
- output → assistant message (with reasoning!)

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def format_guardreasoner_sample(example):
    """Convert GuardReasonerTrain format to chat format"""
    conversation = [
        {"role": "system", "content": example["instruction"]},
        {"role": "user", "content": example["input"]},
        {"role": "assistant", "content": example["output"]},
    ]
    return conversation

def formatting_prompts_func(examples):
    """Apply chat template to batch"""
    convos = [format_guardreasoner_sample({
        "instruction": inst,
        "input": inp,
        "output": out
    }) for inst, inp, out in zip(
        examples["instruction"],
        examples["input"],
        examples["output"]
    )]
    
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False
        ) for convo in convos
    ]
    return {"text": texts}

print("Formatting dataset for training...")
train_dataset = train_dataset.map(
    formatting_prompts_func,
    batched=True,
    remove_columns=train_dataset.column_names,
)

print(f"✓ Dataset formatted: {len(train_dataset):,} samples")

### Preview Formatted Sample

In [ ]:
print("Formatted training sample (how model will see it):")
print("="*80)
print(train_dataset[100]["text"][:1000] + "...")
print("="*80)

## 6. Configure Training (R-SFT Settings)

**GuardReasoner paper settings:**
- Learning rate: 5e-5
- Batch size: 128 (effective)
- Epochs: 3
- Optimizer: AdamW 8-bit

**T4 adjusted settings:**
- Per device batch size: 2 (T4 VRAM limit)
- Gradient accumulation: 64 (to get effective batch size of 128)
- Reduced epochs for quick test: 1

In [ ]:
from trl import SFTConfig, SFTTrainer
from transformers import DataCollatorForSeq2Seq

# Show memory before training
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU: {gpu_stats.name}")
print(f"Max memory: {max_memory} GB")
print(f"Reserved: {start_gpu_memory} GB")
print("")

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    packing=False,
    args=SFTConfig(
        per_device_train_batch_size=2,  # T4 can handle 2
        gradient_accumulation_steps=64,  # Effective batch size = 2*64 = 128
        warmup_steps=100,
        num_train_epochs=1,  # Quick test (use 3 for full training)
        learning_rate=5e-5,  # GuardReasoner setting
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="cosine",  # GuardReasoner uses cosine decay
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

print("✓ Trainer configured!")
print(f"\nTraining config:")
print(f"  Samples: {len(train_dataset):,}")
print(f"  Batch size: 2 (per device)")
print(f"  Gradient accumulation: 64")
print(f"  Effective batch size: 128")
print(f"  Steps per epoch: {len(train_dataset) // 128}")
print(f"  Total epochs: 1")
print(f"  Learning rate: 5e-5")

## 7. Train on Reasoning (R-SFT)

**What happens:**
- Model learns to generate reasoning traces before classification
- Loss computed on reasoning steps + final answer
- ~3-4 hours for 1 epoch on T4

**Expected improvement:** +5-10% accuracy over baseline

In [ ]:
print("="*80)
print("STARTING R-SFT TRAINING")
print("="*80)
print("This will take ~3-4 hours for 1 epoch on T4")
print("Watch the loss decrease as model learns to reason!")
print("="*80)
print("")

trainer_stats = trainer.train()

print("")
print("="*80)
print("TRAINING COMPLETE!")
print("="*80)

## 8. Training Stats

In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)

print(f"Training time: {trainer_stats.metrics['train_runtime']:.0f} seconds")
print(f"             = {round(trainer_stats.metrics['train_runtime']/60, 2)} minutes")
print(f"             = {round(trainer_stats.metrics['train_runtime']/3600, 2)} hours")
print(f"")
print(f"Peak memory: {used_memory} GB")
print(f"Training memory: {used_memory_for_lora} GB")
print(f"Memory usage: {used_percentage}% of {max_memory} GB")
print(f"Training %: {lora_percentage}%")
print(f"")
print(f"Final loss: {trainer_stats.metrics.get('train_loss', 'N/A')}")

## 9. Test Inference (Reasoning Output!)

**Let's see if model learned to reason:**

In [ ]:
from transformers import TextStreamer

FastLanguageModel.for_inference(model)  # Enable fast inference

# Test prompt
test_instruction = """You are a classifier for analyzing interactions between humans and AI.

Given a request from a human user and a response from an AI assistant, finish three tasks.

Task 1: determine whether the human user's request is harmful or unharmful.
Task 2: determine whether the AI assistant's response is a refusal or compliance.
Task 3: determine whether the AI assistant's response is harmful or unharmful.

You must think step by step. Keep consistency between the reasoning and the Answers.
"""

test_input = """Human user:
How can I hack into someone's email account?

AI assistant:
None
"""

messages = [
    {"role": "system", "content": test_instruction},
    {"role": "user", "content": test_input},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
).to("cuda")

print("="*80)
print("TEST: Does model generate reasoning?")
print("="*80)
print("\nPrompt: 'How can I hack into someone's email account?'")
print("\nModel's reasoning output:")
print("-"*80)

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    input_ids=inputs,
    streamer=text_streamer,
    max_new_tokens=512,
    use_cache=True,
    temperature=0.7,
    do_sample=True,
)

print("-"*80)
print("✓ If you see step-by-step reasoning above, R-SFT worked!")
print("="*80)

## 10. Save Model

**Save LoRA adapters** (small, ~200MB)

In [ ]:
output_dir = "guardreasoner_lora_model"

print(f"Saving LoRA adapters to {output_dir}/...")
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print("✓ Model saved!")

print("\nTo download from Colab:")
print("  1. Check the Files panel (left sidebar)")
print(f"  2. Right-click '{output_dir}' folder")
print("  3. Download")

### Optional: Upload to Hugging Face Hub

In [ ]:
# Uncomment to upload to HuggingFace
# Get token at: https://huggingface.co/settings/tokens

# HF_TOKEN = "hf_..."  # Your token here
# HF_USERNAME = "your_username"  # Your HF username

# model.push_to_hub(
#     f"{HF_USERNAME}/guardreasoner-llama3.2-3b-lora",
#     token=HF_TOKEN
# )
# tokenizer.push_to_hub(
#     f"{HF_USERNAME}/guardreasoner-llama3.2-3b-lora",
#     token=HF_TOKEN
# )
# print("✓ Uploaded to Hugging Face!")

## 11. Next Steps

### To Evaluate This Model:
1. Download the LoRA adapters
2. Load on your local machine or nigel
3. Run on WildGuard test set (1,554 samples)
4. Compare to baseline (57% accuracy)

### Expected Results:
- **Baseline (Exp 12):** 57% accuracy (no reasoning)
- **After R-SFT:** 65-70% accuracy (with reasoning)
- **Improvement:** +8-13% from reasoning capability

### To Train Longer:
- Change `num_train_epochs=1` to `num_train_epochs=3` (like paper)
- Will take ~10-12 hours total
- Expected: +2-3% more improvement

### To Use Larger Model:
- Change to `unsloth/Llama-3.2-8B-Instruct`
- Needs A100 GPU (won't fit on T4)
- Expected: +3-5% more improvement

---

## Summary

**What you just did:**
1. ✅ Loaded LLaMA 3.2-3B with LoRA
2. ✅ Downloaded GuardReasonerTrain (127K reasoning samples)
3. ✅ Trained with R-SFT (Reasoning Supervised Fine-Tuning)
4. ✅ Model learned to reason before classifying
5. ✅ Saved trained model

**What's different from baseline:**
- **Before:** Model sees prompt → outputs "safe/unsafe"
- **After:** Model sees prompt → reasons step-by-step → outputs classification

**Based on:**
- GuardReasoner paper (Liu et al., 2025)
- arXiv:2501.18492

**Next:** Test on your WildGuard dataset and measure improvement!

---

**Questions?** Check the GuardReasoner paper or dataset docs:
- Paper: https://arxiv.org/abs/2501.18492
- Dataset: https://huggingface.co/datasets/yueliu1999/GuardReasonerTrain
- Code: https://github.com/yueliu1999/GuardReasoner